**构造xgboost模型，评估数据集难度**

In [6]:
import os
import sys
BASE_DIR = r"C:\Users\nino.zhou\Desktop\my_project\cp_newbank_mobile_behavior"
sys.path.insert(0, BASE_DIR)

import pandas as pd
import numpy as np
from config import setting
from config import setting
import pandas as pd
from xgboost import XGBClassifier
import itertools
from sklearn.externals import joblib
import datetime

In [7]:
df_train = pd.read_csv(setting.BASE_DIR + "\data_model\data\sensor_train.csv")
df_train

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0
...,...,...,...,...,...,...,...,...,...
425354,7291,4561,-0.1,-0.5,-1.7,-0.2,3.4,7.4,18
425355,7291,4647,0.7,-1.9,1.1,0.5,1.7,10.7,18
425356,7291,4735,1.3,-1.4,-0.5,1.4,2.1,8.0,18
425357,7291,4830,1.2,-1.6,-0.1,1.6,1.8,9.2,18


# 1. 特征构造

## 1.1构造是否fragment起点终点的特征

In [8]:
start_mark_list = []
end_mark_list = []
line_num_list = []
last_fragment_id = -1
line_num = 1
for fragment_id in df_train["fragment_id"]:
    if last_fragment_id != fragment_id:
        start_mark_list.append(1)
        end_mark_list.append(1)
        line_num = 1
        line_num_list.append(line_num)
    else:
        start_mark_list.append(0)
        end_mark_list.append(0)
        line_num += 1
        line_num_list.append(line_num)
        
    
    last_fragment_id = fragment_id
end_mark_list.append(1)
end_mark_list = end_mark_list[1:]

In [9]:
len(start_mark_list), len(end_mark_list), len(line_num_list)

(425359, 425359, 425359)

In [10]:
df_train["fragment_start_mark"] = start_mark_list
df_train["fragment_end_mark"] = end_mark_list
df_train["fragment_line_num"] = line_num_list
df_train[50:100]

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,fragment_start_mark,fragment_end_mark,fragment_line_num
50,0,4421,0.0,-0.0,-0.0,0.7,4.7,8.4,0,0,0,51
51,0,4505,-0.0,0.0,-0.1,0.7,4.7,8.5,0,0,0,52
52,0,4596,-0.0,0.1,0.2,0.6,4.7,8.8,0,0,0,53
53,0,4681,-0.1,0.0,-0.0,0.7,4.6,8.5,0,0,0,54
54,0,4768,-0.2,-0.1,0.2,0.9,4.8,8.6,0,0,0,55
55,0,4853,-0.1,-0.1,0.0,0.8,4.7,7.9,0,0,0,56
56,0,4951,0.3,0.1,0.1,0.8,4.7,8.6,0,0,1,57
57,1,58,-0.1,-0.0,0.3,0.8,5.1,8.9,0,1,0,1
58,1,146,0.1,-0.0,0.0,0.8,5.0,8.4,0,0,0,2
59,1,233,0.0,0.1,0.3,0.9,4.9,8.3,0,0,0,3


In [11]:
df_train[["fragment_id", "fragment_start_mark", "fragment_end_mark", "fragment_line_num"]][-100:-50]

,fragment_id,fragment_start_mark,fragment_end_mark,fragment_line_num
425259,7290,0,0,17
425260,7290,0,0,18
425261,7290,0,0,19
425262,7290,0,0,20
425263,7290,0,0,21
425264,7290,0,0,22
425265,7290,0,0,23
425266,7290,0,0,24
425267,7290,0,0,25
425268,7290,0,0,26


## 1.2构造time_point差分特征

In [12]:
df_train["time_point_diff"] = df_train["time_point"].diff().fillna(0)
df_train

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,fragment_start_mark,fragment_end_mark,fragment_line_num,time_point_diff
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0,1,0,1,0.0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0,0,0,2,81.0
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0,0,0,3,90.0
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0,0,0,4,99.0
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0,0,0,5,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425354,7291,4561,-0.1,-0.5,-1.7,-0.2,3.4,7.4,18,0,0,54,84.0
425355,7291,4647,0.7,-1.9,1.1,0.5,1.7,10.7,18,0,0,55,86.0
425356,7291,4735,1.3,-1.4,-0.5,1.4,2.1,8.0,18,0,0,56,88.0
425357,7291,4830,1.2,-1.6,-0.1,1.6,1.8,9.2,18,0,0,57,95.0


# 2. 特征衍生

## 2.1 加减乘除

In [13]:
to_derive_column_list = [column for column in df_train.columns if column not in ["fragment_id", "time_point", "behavior_id"]]
to_derive_column_list

['acc_x',
 'acc_y',
 'acc_z',
 'acc_xg',
 'acc_yg',
 'acc_zg',
 'fragment_start_mark',
 'fragment_end_mark',
 'fragment_line_num',
 'time_point_diff']

In [14]:
import itertools
column_sets = itertools.product(to_derive_column_list, to_derive_column_list)
for col1, col2 in column_sets:
    print(col1, col2)
    df_train[col1 + ":+:" + col2] = df_train[col1] + df_train[col2]
    df_train[col1 + ":-:" + col2] = df_train[col1] - df_train[col2]
    df_train[col1 + ":*:" + col2] = df_train[col1] * df_train[col2]
    df_train[col1 + ":/:" + col2] = df_train[col1] / (df_train[col2] + 10)

acc_x acc_x
acc_x acc_y
acc_x acc_z
acc_x acc_xg
acc_x acc_yg
acc_x acc_zg
acc_x fragment_start_mark
acc_x fragment_end_mark
acc_x fragment_line_num
acc_x time_point_diff
acc_y acc_x
acc_y acc_y
acc_y acc_z
acc_y acc_xg
acc_y acc_yg
acc_y acc_zg
acc_y fragment_start_mark
acc_y fragment_end_mark
acc_y fragment_line_num
acc_y time_point_diff
acc_z acc_x
acc_z acc_y
acc_z acc_z
acc_z acc_xg
acc_z acc_yg
acc_z acc_zg
acc_z fragment_start_mark
acc_z fragment_end_mark
acc_z fragment_line_num
acc_z time_point_diff
acc_xg acc_x
acc_xg acc_y
acc_xg acc_z
acc_xg acc_xg
acc_xg acc_yg
acc_xg acc_zg
acc_xg fragment_start_mark
acc_xg fragment_end_mark
acc_xg fragment_line_num
acc_xg time_point_diff
acc_yg acc_x
acc_yg acc_y
acc_yg acc_z
acc_yg acc_xg
acc_yg acc_yg
acc_yg acc_zg
acc_yg fragment_start_mark
acc_yg fragment_end_mark
acc_yg fragment_line_num
acc_yg time_point_diff
acc_zg acc_x
acc_zg acc_y
acc_zg acc_z
acc_zg acc_xg
acc_zg acc_yg
acc_zg acc_zg
acc_zg fragment_start_mark
acc_zg fragment_e

In [15]:
df_train

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,fragment_start_mark,...,time_point_diff:*:fragment_end_mark,time_point_diff:/:fragment_end_mark,time_point_diff:+:fragment_line_num,time_point_diff:-:fragment_line_num,time_point_diff:*:fragment_line_num,time_point_diff:/:fragment_line_num,time_point_diff:+:time_point_diff,time_point_diff:-:time_point_diff,time_point_diff:*:time_point_diff,time_point_diff:/:time_point_diff
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0,1,...,0.0,0.000000,1.0,-1.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0,0,...,0.0,8.100000,83.0,79.0,162.0,6.750000,162.0,0.0,6561.0,0.890110
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0,0,...,0.0,9.000000,93.0,87.0,270.0,6.923077,180.0,0.0,8100.0,0.900000
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0,0,...,0.0,9.900000,103.0,95.0,396.0,7.071429,198.0,0.0,9801.0,0.908257
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0,0,...,0.0,9.100000,96.0,86.0,455.0,6.066667,182.0,0.0,8281.0,0.900990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425354,7291,4561,-0.1,-0.5,-1.7,-0.2,3.4,7.4,18,0,...,0.0,8.400000,138.0,30.0,4536.0,1.312500,168.0,0.0,7056.0,0.893617
425355,7291,4647,0.7,-1.9,1.1,0.5,1.7,10.7,18,0,...,0.0,8.600000,141.0,31.0,4730.0,1.323077,172.0,0.0,7396.0,0.895833
425356,7291,4735,1.3,-1.4,-0.5,1.4,2.1,8.0,18,0,...,0.0,8.800000,144.0,32.0,4928.0,1.333333,176.0,0.0,7744.0,0.897959
425357,7291,4830,1.2,-1.6,-0.1,1.6,1.8,9.2,18,0,...,0.0,9.500000,152.0,38.0,5415.0,1.417910,190.0,0.0,9025.0,0.904762


## 2.2 均线

In [16]:
def create_mean_feature(df_groupby, col, day):
    return df_groupby[col].rolling(day).mean()

for col in to_derive_column_list:
    for day in [2,4,6,8,10]:
        df_train[col + "_{}_mean".format(day)] = df_train[["fragment_id", col]].groupby("fragment_id").apply(create_mean_feature, col, day).reset_index(drop=True)

In [17]:
df_train = df_train.fillna(0)
df_train.shape

(425359, 463)

# 切分训练验证测试集

In [18]:
# def random_train_test_split(df_train):
#     from sklearn.model_selection import train_test_split
#     X = df_train.drop("behavior_id", axis=1)
#     y = df_train["behavior_id"]
#     X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.4, random_state=10086)
#     X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=10087)
#     return X_train, X_val, y_train, y_val, X_test, y_test
# X_train, X_val, y_train, y_val, X_test, y_test = random_train_test_split(df_train)
# X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape, y_test.shape

In [19]:
# # fragment_id随机替换
# unique_fragment_ids = list(df_train["fragment_id"].drop_duplicates())
# new_unique_fragment_ids = np.random.choice(a=range(len(unique_fragment_id)), size=(len(unique_fragment_id),), replace=False)
# pair_dict = {old_id: new_id for old_id, new_id in zip(unique_fragment_ids, new_unique_fragment_ids)}
# df_train["fragment_id"] = df_train["fragment_id"].apply(lambda x: 100 * pair_dict[x])

In [20]:
# def ramdom_df(df):
#     """随机打乱df数据的顺序"""
#     random_list = list(df.index)
#     np.random.shuffle(random_list)
#     return df.loc[random_list]

In [21]:
# 构造行号（因为元数据就是按顺序排列，因此不需重置顺序）
df_train = df_train.reset_index()
df_train = df_train.rename(columns={"index": "id"})
df_train

,id,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,...,fragment_line_num_2_mean,fragment_line_num_4_mean,fragment_line_num_6_mean,fragment_line_num_8_mean,fragment_line_num_10_mean,time_point_diff_2_mean,time_point_diff_4_mean,time_point_diff_6_mean,time_point_diff_8_mean,time_point_diff_10_mean
0,0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000,0.0
1,1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0,...,1.5,0.0,0.0,0.0,0.0,40.5,0.00,0.000000,0.000,0.0
2,2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0,...,2.5,0.0,0.0,0.0,0.0,85.5,0.00,0.000000,0.000,0.0
3,3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0,...,3.5,2.5,0.0,0.0,0.0,94.5,67.50,0.000000,0.000,0.0
4,4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0,...,4.5,3.5,0.0,0.0,0.0,95.0,90.25,0.000000,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425354,425354,7291,4561,-0.1,-0.5,-1.7,-0.2,3.4,7.4,18,...,53.5,52.5,51.5,50.5,49.5,88.0,86.25,85.666667,85.750,85.6
425355,425355,7291,4647,0.7,-1.9,1.1,0.5,1.7,10.7,18,...,54.5,53.5,52.5,51.5,50.5,85.0,86.50,85.833333,85.500,85.7
425356,425356,7291,4735,1.3,-1.4,-0.5,1.4,2.1,8.0,18,...,55.5,54.5,53.5,52.5,51.5,87.0,87.50,86.500000,86.000,86.0
425357,425357,7291,4830,1.2,-1.6,-0.1,1.6,1.8,9.2,18,...,56.5,55.5,54.5,53.5,52.5,91.5,88.25,88.166667,87.250,86.7


In [22]:
# 因为要模型合并，因此各模型必须有统一的测试集，统一使用按lstm模型切分的测试集作为xgboost和biresnet的测试集
arr_fragment_ids = np.load(os.path.join(setting.BASE_DIR, "./data_model/data/lstm_arr_fragment_ids.npy"))
test_fragment_ids = list(arr_fragment_ids[5000:])
len(test_fragment_ids)

2292

In [23]:
def fragment_random_train_test_split(df_train, test_fragment_ids):
    """按照fragment_id作为基础单元来切分数据集"""    
    
    from sklearn.model_selection import train_test_split
    unique_fragment_id = list(df_train["fragment_id"].drop_duplicates())  # 其实也没必要去重，因为数据集的fragment_id都是唯一的
    
    X_train_val_ids = [fragment_id for fragment_id in unique_fragment_id if fragment_id not in test_fragment_ids]
    X_test_ids = test_fragment_ids
    X_train_ids, X_val_ids = train_test_split(X_train_val_ids, test_size=0.2, random_state=10087)
    
    df_list = [df_train[df_train["fragment_id"] == fragment_id] for fragment_id in X_train_ids]  # 确保即打乱顺序，又不破坏同fragment_id数据连续性
    df_train_dataset = pd.concat(df_list, ignore_index=True)
    df_train_dataset = df_train_dataset.drop(["id", "fragment_id"], axis=1)  # 此处应当丢弃，因为万一测试集被随机成同一个fragment_id，将导致分类错误
#     df_train_dataset = ramdom_df(df_train_dataset)  # 因为X_train_ids已经是随机打乱的，此处不应当把时序也随机打乱
    X_train = df_train_dataset.drop("behavior_id", axis=1)
    y_train = df_train_dataset["behavior_id"]
    
    df_list = [df_train[df_train["fragment_id"] == fragment_id] for fragment_id in X_val_ids]
    df_val_dataset = pd.concat(df_list, ignore_index=True)
    df_val_dataset = df_val_dataset.drop(["id", "fragment_id"], axis=1)
    X_val = df_val_dataset.drop("behavior_id", axis=1)
    y_val = df_val_dataset["behavior_id"]
    
    df_list = [df_train[df_train["fragment_id"] == fragment_id] for fragment_id in X_test_ids]
    df_test_dataset = pd.concat(df_list, ignore_index=True)
    test_index = list(df_test_dataset["id"])  # 保存测试集行索引，因为后面要根据索引，报错对于样本的测试集预测结果
    df_test_dataset = df_test_dataset.drop(["id", "fragment_id"], axis=1)
    X_test = df_test_dataset.drop("behavior_id", axis=1)
    y_test = df_test_dataset["behavior_id"]
    

    return X_train, X_val, y_train, y_val, X_test, y_test, test_index
X_train, X_val, y_train, y_val, X_test, y_test, test_index = fragment_random_train_test_split(df_train, test_fragment_ids)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape, y_test.shape

((233215, 461), (58395, 461), (233215,), (58395,), (133749, 461), (133749,))

In [24]:
X_train[X_train.columns[:15]]

,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,fragment_start_mark,fragment_end_mark,fragment_line_num,time_point_diff,acc_x:+:acc_x,acc_x:-:acc_x,acc_x:*:acc_x,acc_x:/:acc_x
0,65,0.0,-0.1,-0.0,9.2,-0.6,3.4,1,0,1,-4918.0,0.0,0.0,0.00,0.000000
1,149,-0.0,0.1,-0.2,9.2,-0.4,3.3,0,0,2,84.0,-0.0,0.0,0.00,-0.000000
2,233,-0.1,0.1,0.2,9.1,-0.5,3.7,0,0,3,84.0,-0.2,0.0,0.01,-0.010101
3,316,-0.0,0.1,-0.1,9.2,-0.5,3.4,0,0,4,83.0,-0.0,0.0,0.00,-0.000000
4,399,0.0,0.1,-0.0,9.2,-0.5,3.4,0,0,5,83.0,0.0,0.0,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233210,4655,0.0,-0.0,-0.0,8.9,-0.4,4.2,0,0,56,84.0,0.0,0.0,0.00,0.000000
233211,4738,0.0,0.1,-0.0,8.9,-0.4,4.1,0,0,57,83.0,0.0,0.0,0.00,0.000000
233212,4821,0.0,-0.0,0.0,8.9,-0.4,4.2,0,0,58,83.0,0.0,0.0,0.00,0.000000
233213,4905,-0.0,-0.0,0.0,8.9,-0.5,4.2,0,0,59,84.0,-0.0,0.0,0.00,-0.000000


# 模型构造与训练

In [25]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(objective="multi:softmax", num_class=19, n_jobs=4)

xgb_clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mlogloss', early_stopping_rounds=10)

[0]	validation_0-mlogloss:2.23232	validation_1-mlogloss:2.29492
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 10 rounds.
[1]	validation_0-mlogloss:1.97937	validation_1-mlogloss:2.06767
[2]	validation_0-mlogloss:1.81619	validation_1-mlogloss:1.91880
[3]	validation_0-mlogloss:1.69309	validation_1-mlogloss:1.80455
[4]	validation_0-mlogloss:1.59937	validation_1-mlogloss:1.72201
[5]	validation_0-mlogloss:1.52193	validation_1-mlogloss:1.65406
[6]	validation_0-mlogloss:1.45811	validation_1-mlogloss:1.59909
[7]	validation_0-mlogloss:1.40465	validation_1-mlogloss:1.55243
[8]	validation_0-mlogloss:1.35638	validation_1-mlogloss:1.51300
[9]	validation_0-mlogloss:1.31785	validation_1-mlogloss:1.48259
[10]	validation_0-mlogloss:1.28158	validation_1-mlogloss:1.45322
[11]	validation_0-mlogloss:1.24909	validation_1-mlogloss:1.42847
[12]	validation_0-mlogloss:1.21776	validation_1-mlogloss:1.4064

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_class=19, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## 输出特征重要性

In [26]:
importance_dict = {col: importance for col, importance in zip(X_train.columns, xgb_clf.feature_importances_)}
sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

[('acc_y:*:acc_y', 0.061216485),
 ('acc_xg:*:acc_xg', 0.03498358),
 ('acc_yg:/:acc_zg', 0.032527357),
 ('acc_x:*:acc_x', 0.032142542),
 ('acc_yg:*:acc_yg', 0.030922491),
 ('acc_x:-:acc_zg', 0.02916937),
 ('acc_xg:*:acc_zg', 0.023640277),
 ('acc_yg:/:acc_y', 0.022976737),
 ('acc_yg:+:acc_zg', 0.022420347),
 ('acc_z:-:acc_zg', 0.020622944),
 ('acc_y:-:acc_yg', 0.016368242),
 ('acc_xg:/:acc_yg', 0.015552676),
 ('acc_yg:/:acc_xg', 0.014976409),
 ('acc_y:-:acc_xg', 0.014799652),
 ('acc_zg:/:acc_xg', 0.014674055),
 ('acc_yg:*:acc_zg', 0.014615045),
 ('acc_xg:-:acc_yg', 0.0145984795),
 ('acc_xg:+:acc_zg', 0.014296232),
 ('acc_xg:+:acc_yg', 0.01220072),
 ('acc_z:*:acc_z', 0.012116155),
 ('acc_xg:/:acc_zg', 0.011749432),
 ('acc_x:-:acc_xg', 0.010196101),
 ('acc_xg:*:acc_yg', 0.0096815815),
 ('acc_xg:/:acc_x', 0.009106987),
 ('time_point_diff_4_mean', 0.009017985),
 ('acc_xg:-:acc_zg', 0.008792144),
 ('acc_y_10_mean', 0.008462343),
 ('acc_zg:/:acc_yg', 0.008267408),
 ('acc_yg', 0.008262828),
 ('

## 模型保存

In [27]:
from sklearn.externals import joblib
import datetime
joblib.dump(xgb_clf, setting.BASE_DIR + "/data_model/model/xgboost_{}.pkl".format(datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d")))

['C:\\Users\\nino.zhou\\Desktop\\my_project\\cp_newbank_mobile_behavior/data_model/model/xgboost_2020-07-05.pkl']

# 模型评估

In [28]:
y_pred = xgb_clf.predict(X_test)
y_pred.shape

(133749,)

In [29]:
from sklearn.metrics import accuracy_score
y_true = y_test
accuracy_score(y_true, y_pred)

0.62982900806735

### 保存测试集评估结果

In [31]:
df_train["category_xgboost"] = None
df_train.loc[test_index, "category_xgboost"] = y_pred
df_train = df_train[["id", "fragment_id", "time_point", "behavior_id", "category_xgboost"]]
df_train.to_csv(setting.BASE_DIR + "\data_model\data\category_xgboost.csv")
df_train

C:\Users\nino.zhou\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\nino.zhou\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,id,fragment_id,time_point,behavior_id,category_xgboost
0,0,0,27,0,None
1,1,0,108,0,None
2,2,0,198,0,None
3,3,0,297,0,None
4,4,0,388,0,None
...,...,...,...,...,...
425354,425354,7291,4561,18,None
425355,425355,7291,4647,18,None
425356,425356,7291,4735,18,None
425357,425357,7291,4830,18,None


## 加载官方测试集评估

In [29]:
df_tsst = pd.read_csv(setting.BASE_DIR + "\data_model\data\sensor_test.csv")
df_tsst

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg
0,0,230,-1.3,0.4,0.5,-1.0,6.9,6.3
1,0,318,-1.2,0.8,1.0,-0.6,7.4,7.8
2,0,403,0.0,-0.1,-0.7,0.6,6.7,6.2
3,0,490,0.9,-0.1,0.1,1.6,6.5,6.9
4,0,574,0.4,0.1,-0.4,0.9,6.6,6.6
...,...,...,...,...,...,...,...,...
466887,7999,4618,3.6,-5.7,11.5,9.4,-12.3,7.9
466888,7999,4702,5.1,-0.7,5.5,9.0,-8.5,1.7
466889,7999,4787,2.7,1.5,4.8,6.3,-6.8,2.0
466890,7999,4874,2.0,1.4,4.6,7.0,-6.9,2.4


In [3]:
from xgboost_model.main import CreateFeature, XgboostModeling

obj = CreateFeature()
df_test = obj.load_test_sample()
df_test = obj.create_feature(df_test)
df_test = obj.derive_feature(df_test)

C:\Users\nino.zhou\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


acc_x acc_x
acc_x acc_y
acc_x acc_z
acc_x acc_xg
acc_x acc_yg
acc_x acc_zg
acc_x fragment_start_mark
acc_x fragment_end_mark
acc_x fragment_line_num
acc_x time_point_diff
acc_y acc_x
acc_y acc_y
acc_y acc_z
acc_y acc_xg
acc_y acc_yg
acc_y acc_zg
acc_y fragment_start_mark
acc_y fragment_end_mark
acc_y fragment_line_num
acc_y time_point_diff
acc_z acc_x
acc_z acc_y
acc_z acc_z
acc_z acc_xg
acc_z acc_yg
acc_z acc_zg
acc_z fragment_start_mark
acc_z fragment_end_mark
acc_z fragment_line_num
acc_z time_point_diff
acc_xg acc_x
acc_xg acc_y
acc_xg acc_z
acc_xg acc_xg
acc_xg acc_yg
acc_xg acc_zg
acc_xg fragment_start_mark
acc_xg fragment_end_mark
acc_xg fragment_line_num
acc_xg time_point_diff
acc_yg acc_x
acc_yg acc_y
acc_yg acc_z
acc_yg acc_xg
acc_yg acc_yg
acc_yg acc_zg
acc_yg fragment_start_mark
acc_yg fragment_end_mark
acc_yg fragment_line_num
acc_yg time_point_diff
acc_zg acc_x
acc_zg acc_y
acc_zg acc_z
acc_zg acc_xg
acc_zg acc_yg
acc_zg acc_zg
acc_zg fragment_start_mark
acc_zg fragment_e

In [4]:
df_test.shape

(466892, 462)

In [11]:
df_test

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,fragment_start_mark,fragment_end_mark,...,fragment_line_num_2_mean,fragment_line_num_4_mean,fragment_line_num_6_mean,fragment_line_num_8_mean,fragment_line_num_10_mean,time_point_diff_2_mean,time_point_diff_4_mean,time_point_diff_6_mean,time_point_diff_8_mean,time_point_diff_10_mean
0,0,230,-1.3,0.4,0.5,-1.0,6.9,6.3,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000,0.0
1,0,318,-1.2,0.8,1.0,-0.6,7.4,7.8,0,0,...,1.5,0.0,0.0,0.0,0.0,44.0,0.00,0.000000,0.000,0.0
2,0,403,0.0,-0.1,-0.7,0.6,6.7,6.2,0,0,...,2.5,0.0,0.0,0.0,0.0,86.5,0.00,0.000000,0.000,0.0
3,0,490,0.9,-0.1,0.1,1.6,6.5,6.9,0,0,...,3.5,2.5,0.0,0.0,0.0,86.0,65.00,0.000000,0.000,0.0
4,0,574,0.4,0.1,-0.4,0.9,6.6,6.6,0,0,...,4.5,3.5,0.0,0.0,0.0,85.5,86.00,0.000000,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466887,7999,4618,3.6,-5.7,11.5,9.4,-12.3,7.9,0,0,...,54.5,53.5,52.5,51.5,50.5,86.5,85.50,85.000000,85.625,85.4
466888,7999,4702,5.1,-0.7,5.5,9.0,-8.5,1.7,0,0,...,55.5,54.5,53.5,52.5,51.5,84.0,85.25,85.000000,85.375,85.4
466889,7999,4787,2.7,1.5,4.8,6.3,-6.8,2.0,0,0,...,56.5,55.5,54.5,53.5,52.5,84.5,85.50,85.166667,84.875,85.4
466890,7999,4874,2.0,1.4,4.6,7.0,-6.9,2.4,0,0,...,57.5,56.5,55.5,54.5,53.5,86.0,85.00,85.500000,85.250,85.5


In [9]:
xgb_clf = joblib.load(setting.BASE_DIR + "/data_model/model/xgboost_{}.pkl".format(
    datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d")))

In [13]:
X = df_test.values

In [14]:
y_pred = xgb_clf.predict(X)
y_pred

array([ 0,  0,  0, ..., 18, 18, 18], dtype=int64)

In [16]:
y_pred[:50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [17]:
y_pred[50:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [18]:
y_pred[100:150]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [19]:
y_pred[150:200]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [20]:
y_pred[1000:1050]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)